In [263]:
import bs4
import requests
from urllib.request import urlopen
import pandas as pd

In [264]:
class Content:
    def __init__(self, outlet, topic, url, title, author, date, body):
        self.outlet = outlet
        self.topic =topic
        self.title = title
        self.author = author
        self.date = date
        self.body = body
        self.url = url
        
    def print(self):
        print("Article found for topic: {}".format(self.topic))
        print("Outlet: {}".format(self.outlet))
        print("Url: {}".format(self.url))
        print("Title: {}".format(self.title))
        print("Author: {}".format(self.author))
        print("Date: {}".format(self.date))
        print("Body: {}".format(self.body))
        
    def makelist(self):
        mylist = [self.topic, self.outlet, self.url, self.title, self.author, self.date, self.body]
        return pd.Series(mylist)
        
              

In [265]:
class Website:
    def __init__(self, name, url, searchUrl, resultListing, resultUrl, absoluteUrl, 
                 titleTag, authorTag, dateTag, bodyTag):
        self.name = name
        self.url = url
        self.searchUrl = searchUrl
        self.resultListing = resultListing
        self.resultUrl = resultUrl
        self.absoluteUrl = absoluteUrl
        self.titleTag = titleTag
        self.authorTag = authorTag
        self.dateTag = dateTag
        self.bodyTag = bodyTag
        

In [303]:
class Crawler:
    
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return bs4.BeautifulSoup(req.text, 'html.parser')
    
    def safeGet(self, pageObj, selector):
        childObj = pageObj.select(selector)
        if childObj is not None and len(childObj) >0:
            s = ""
            for i in range(len(childObj)):
                s=s+ childObj[i].get_text()
            return s
        return ""
    
    def search(self, paper, topic, site):
        bs = self.getPage(site.searchUrl)
        searchResults = bs.select(site.resultListing)
        mylist = []
        for result in searchResults:
            url = result.select(site.resultUrl)[0].a.attrs["href"]
            print(url)
            if (site.absoluteUrl):
                bs.self.getPage(url)
            else:
                bs = self.getPage(site.url+url)
            if bs is None:
                return
            title = self.safeGet(bs, site.titleTag)
            author = self.safeGet(bs, site.authorTag)
            date = self.safeGet(bs, site.dateTag)
            body = self.safeGet(bs, site.bodyTag)
            if title !='':
                content = Content(paper, topic, url, title, author, date, body)
                #content.print()
                mylist.append(content.makelist())
        return mylist
                

In [304]:
crawler = Crawler()

In [311]:
siteData = []
for i in range(10):
    siteData.append(['msnbc', "http://www.msnbc.com", 
             "http://www.msnbc.com/search/gun?sm_field_issues=&sm_field_show=&date[min]&date[max]&date[date_selector]=&page="+str(i)+"&f[0]=bundle%3Aarticle", 
             "div.search-result.article", "h3.title.search-result__teaser__title", False,
              "h1.is-title-pane.panel-pane.pane-node-title", "span.author", 
             "time.article-date-posted", "div.pane-content"])

In [312]:
sites = []
for row in siteData:
    sites.append(Website(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9]))

In [313]:
topics = ['gun']
source = 'msnbc'
output = []
for topic in topics:
    for targetSite in sites:
        out = crawler.search(topic, source, targetSite)

/rachel-maddow-show/roe-v-wade-may-be-trouble-it-enjoys-broad-public-support
/rachel-maddow-show/facebook-posts-lead-call-democratic-candidate-dump-his-campaign-chief-staff
/rachel-maddow-show/mondays-campaign-round-62518
/rachel-maddow-show/trumps-reverse-midas-touch-extends-attitudes-immigration
/rachel-maddow-show/interiors-zinke-confronts-new-halliburton-related-controversy
/morning-joe/excerpt-neveragain
/rachel-maddow-show/tuesdays-campaign-round-61918
/rachel-maddow-show/mondays-mini-report-61818
/rachel-maddow-show/why-florida-failed-do-concealed-weapons-background-checks
/rachel-maddow-show/white-houses-commission-school-safety-will-ignore-guns
/rachel-maddow-show/mondays-mini-report-6418
/rachel-maddow-show/mondays-campaign-round-6418
/rachel-maddow-show/grieving-mother-after-meeting-trump-it-was-talking-toddler
/rachel-maddow-show/asked-about-school-shootings-sanders-points-dubious-commission
/rachel-maddow-show/wednesdays-mini-report-53018
/rachel-maddow-show/gop-rep-connec

/rachel-maddow-show/mondays-mini-report-11617-0
/rachel-maddow-show/following-mass-shooting-trump-says-isnt-guns-situation
/morning-joe/excerpt-republican-me-0
/rachel-maddow-show/gop-rep-tackling-bump-stocks-would-be-perversion-party-agenda
/rachel-maddow-show/fridays-campaign-round-101317
/rachel-maddow-show/asked-about-mass-shootings-gop-senator-advises-get-small
/rachel-maddow-show/the-problem-paul-ryans-focus-mental-health-and-guns
/rachel-maddow-show/white-house-gets-selective-about-its-post-violence-standards
/rachel-maddow-show/congressional-republicans-still-back-looser-gun-restrictions
/rachel-maddow-show/the-worst-mass-shooting-modern-american-history
/rachel-maddow-show/wednesdays-mini-report-92017
/rachel-maddow-show/british-officials-admonish-trump-after-unhelpful-terror-tweets
/rachel-maddow-show/the-nras-backing-gun-silencer-bill-gets-attention-house
/rachel-maddow-show/epa-chief-its-insensitive-mention-climate-change-right-now
/rachel-maddow-show/new-evidence-emerges-h

In [318]:
df = pd.DataFrame(out)
print(df)

        0    1                                                  2  \
0   msnbc  gun  /rachel-maddow-show/congressional-republicans-...   
1   msnbc  gun  /rachel-maddow-show/the-worst-mass-shooting-mo...   
2   msnbc  gun   /rachel-maddow-show/wednesdays-mini-report-92017   
3   msnbc  gun  /rachel-maddow-show/british-officials-admonish...   
4   msnbc  gun  /rachel-maddow-show/the-nras-backing-gun-silen...   
5   msnbc  gun  /rachel-maddow-show/epa-chief-its-insensitive-...   
6   msnbc  gun  /rachel-maddow-show/new-evidence-emerges-how-r...   
7   msnbc  gun     /rachel-maddow-show/tuesdays-mini-report-82917   
8   msnbc  gun   /rachel-maddow-show/tuesdays-campaign-round-8817   
9   msnbc  gun  /rachel-maddow-show/the-nras-core-message-take...   
10  msnbc  gun  /rachel-maddow-show/russia-scandal-presents-co...   
11  msnbc  gun  /rachel-maddow-show/the-nra-finds-new-ways-sto...   
12  msnbc  gun  /rachel-maddow-show/dc-area-shooting-delays-co...   
13  msnbc  gun   /rachel-maddow-sh

In [317]:
print(len(df))

15
